#### Group Name : Binary Brains

    1. Alisha Momin - rmu993
    2. Anbumani Dhanapal - mot356
    3. Aqdas Hussain - whu947
    4. Sreeja Yalamaddi - yxm265

#### CSV Generation for 50 sample comments

In [1]:
import json
import csv
import numpy as np

with open('small_sample(1).json', 'r') as file:
    content = file.read()

outfile =  open('sample_comments_50.csv', 'w',newline = '')
outputwriter = csv.writer(outfile)
outputwriter.writerow(['Comment','Technology','Technology in Education','Technology in Workforce Development'])
json_objects = content.splitlines()
comments_list = []
count = 0
for obj in json_objects:
    data = json.loads(obj) 
    val = {'Comment': data['body']}
    if val in comments_list:
        pass
    else:
        count = count+1
        comments_list.append({'Comment': data['body']})
        outputwriter.writerow([data['body']])
    if count == 50:
        break
outfile.close()

#### Calculate Fleiss Kappa and Identify disagreements

In [2]:
def read_annotations(files, category):
    annotations = {}
    for file in files:
        with open(file, mode='r', newline='') as f:
            reader = csv.DictReader(f)
            for row in reader:
                comment = row['Comment']
                annotation = int(row[category]) 
                
                if comment not in annotations:
                    annotations[comment] = []
                annotations[comment].append(annotation)
        annotation_list = []
    for _, annotation_values in annotations.items():
        annotation_list.append(annotation_values)
    
    return annotation_list

def fleiss_kappa(annotation_list):
    kappa_matrix = []
    for annotation in annotation_list:
        count_0 = annotation.count(0)
        count_1 = annotation.count(1)
        kappa_matrix.append([count_0, count_1]) 
    kappa_matrix = np.array(kappa_matrix)
    N = kappa_matrix.shape[0]
    n_annotators = np.sum(kappa_matrix[0])
    p_j = kappa_matrix.sum(axis=0) / (N * n_annotators)
    P_i = (np.sum(kappa_matrix * (kappa_matrix - 1), axis=1)) / (n_annotators * (n_annotators - 1))
    P_bar = np.mean(P_i)
    P_e = np.sum(p_j ** 2)
    kappa = (P_bar - P_e) / (1 - P_e)
    return kappa


def identify_major_disagreement(annotation_list, threshold=0.5):
    disagreed_comments = []
    
    for i, annotation_values in enumerate(annotation_list):
        count_0 = annotation_values.count(0)
        count_1 = annotation_values.count(1)
        
        total_annotators = len(annotation_values)
        disagreement_ratio = abs(count_0 - count_1) / total_annotators
        
        if disagreement_ratio < threshold:
            disagreed_comments.append((i, count_0, count_1)) 
    
    return disagreed_comments


In [18]:
files = ["Reddit_comments_labeling_others.csv","sample_comments_50_updated.csv"]
label_categories = ['Technology', 'Technology in Education', 'Technology in Workforce Development']

for category in label_categories:
    annotation_list = read_annotations(files, category)
    kappa = fleiss_kappa(annotation_list)
    print(f"Fleiss' Kappa for '{category}': {kappa}")
#     disagreed_comments = identify_major_disagreement(annotation_list, threshold=0.5)
#     if disagreed_comments:
#         print("Comments with major disagreement:")
#         for i, count_0, count_1 in disagreed_comments:
#             print(f"Comment Index: {i} | 0's: {count_0} | 1's: {count_1}")
#     else:
#         print("No major disagreements.")
#     indexes = [value[0] for value in disagreed_comments]
#     if len(indexes):
#         disagreed_comments = [comments_list[i] for i in indexes]
#         print(category)

Fleiss' Kappa for 'Technology': 0.7802197802197802
Fleiss' Kappa for 'Technology in Education': 0.6563573883161511
Fleiss' Kappa for 'Technology in Workforce Development': 0.6563573883161511


In [3]:
import csv
import json

#### CSV Generation for 950 sample comments

In [260]:
with open('small_sample(1).json', 'r', encoding='utf-8') as file:
    content = file.read()

with open('sample_comments_950.csv', 'w', newline='', encoding='utf-8') as outfile:
    outputwriter = csv.writer(outfile)
    outputwriter.writerow(['Comment', 'Technology', 'Technology in Education', 'Technology in Workforce Development'])
    json_objects = content.splitlines()
    comments_set = set()
    count = 0
    for obj in json_objects[:]:
        try:
            data = json.loads(obj)
            comment = data['body']
            if comment not in comments_set:
                comments_set.add(comment)
                len(comment) > 50
                count += 1
                outputwriter.writerow([comment, '', '', '']) 
#             if count == 950:
#                 break

        except json.JSONDecodeError:
            print(f"Skipping invalid JSON object: {obj}")  


In [261]:
import pandas as pd
data_final = pd.read_csv('sample_comments_950.csv')

In [262]:
data_final['comment_len'] = data_final['Comment'].apply(lambda x : len(x.split()))

In [263]:
data_final['web_link'] = data_final['Comment'].str.contains('http', case=False, na=False).astype(int)

In [293]:
# Expanded tech-related lexicon including network/internet jargon
tech_lexicon_expanded = [
    "python", "java", "javascript", "ai", "artificial intelligence", "machine learning", "deep learning", "cloud", "iot",
    "raspberry pi", "arduino", "robotics", "blockchain", "internet of things", "technology", "software", "hardware", "programming",
    "coding", "app", "application", "android", "iphone", "linux", "github", "developer", "open source", "website", "cloud computing",
    "api", "devops", "big data", "database", "data science", "cloud storage", "virtualization", "datacenter", "server", "network",
    "fiber", "microsoft", "apple", "amazon", "uber", "lyft", "grubhub", "ubereats", "doordash", "instacart", "walmart", "ebay",
    "shopify", "airbnb", "zoom", "slack", "docker", "kubernetes", "dev", "scripting", "automation", "tech startup", "gadget", "smartphone",
    "iot device", "raspberry pi", "software engineer", "developer tools", "sdk", "cybersecurity", "data", "analytics", "big data",
    "tcp/ip", "udp", "ftp", "dns", "ssl", "tls", "ip address", "ipv4", "ipv6", "ssh", "snmp", "bgp", "dhcp", "nat",
    "lan", "wan", "router", "switch", "hub", "gateway", "firewall", "load balancer", "vpn", "proxy server", "access point", "wifi",
    "bluetooth", "4g", "5g", "ethernet", "broadband", "fiber optic", "lte", "mobile hotspot", "data plan", "bandwidth", "latency",
    "ping", "cloud services", "aws", "google cloud", "azure", "cloud storage", "google drive", "dropbox", "icloud", "cloud computing",
    "edge computing", "cdn", "firewall", "encryption", "cybersecurity", "malware", "phishing", "ddos", "data breach", "two-factor authentication",
    "ssl certificate", "cyberattack", "hacking", "intrusion detection", "packet sniffing", "security protocol", "ISP", "fiber optic internet",
    "internet speed", "bandwidth throttling", "network throughput", "routing", "switching", "network topology", "vlan", "subnet", "dns resolution",
    "traceroute", "throughput", "bandwidth usage", "network monitoring", "speed test", "packet loss", "debugging", "error logs", "network diagnostics",
    "ecommerce", "marketplace", "shopping cart", "payment gateway", "online transactions", "web apps", "mobile apps", "qr code", "digital wallet",
    "streaming", "bandwidth usage", "hd", "uhd", "4k", "netflix","rackspace", "geekdom","youtube", "hulu", "twitch", "buffering", "video quality", "audio streaming"
]
def is_tech_related_expanded(comment, lexicon):
    # Preprocess the comment and split into words
    words = comment.split()
    # Count occurrences of tech-related words from the expanded lexicon
    tech_count = sum(1 for word in words if word in lexicon)
    return tech_count if tech_count > 0 else 0

# Apply the lexicon-based classification with expanded lexicon
data_final['predicted_label'] = data_final['Comment'].apply(lambda x: is_tech_related_expanded(x, tech_lexicon_expanded))


In [299]:
data_final[data_final['predicted_label'] > 0][data_final['comment_len'] > 5][data_final['web_link'] == 1]

<ipython-input-299-b20e2cdc8fc2>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_final[data_final['predicted_label'] > 0][data_final['comment_len'] > 5][data_final['web_link'] == 1]
<ipython-input-299-b20e2cdc8fc2>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_final[data_final['predicted_label'] > 0][data_final['comment_len'] > 5][data_final['web_link'] == 1]


,Comment,Technology,Technology in Education,Technology in Workforce Development,comment_len,web_link,predicted_label
30,I am using selenium to automatically fill in t...,NaN,NaN,NaN,301,1,2
84,There's a weird market shortage of used cars r...,NaN,NaN,NaN,129,1,1
87,Also here's another post with people saying fi...,NaN,NaN,NaN,13,1,1
173,Current SAC student who just transferred in Ja...,NaN,NaN,NaN,164,1,1
253,As someone with a disability I can’t begin to ...,NaN,NaN,NaN,161,1,1
256,&gt; One if the best explanations I've seen in...,NaN,NaN,NaN,1429,1,1
379,The Tech community in San Antonio has gotten m...,NaN,NaN,NaN,205,1,1
473,Oh am I?\n\n&gt;Although it can handicap a job...,NaN,NaN,NaN,335,1,2
547,"&gt; Saying ""don't bother"" is a surefire way t...",NaN,NaN,NaN,282,1,2
591,Here are the current closures. You can get upd...,NaN,NaN,NaN,35,1,1


In [204]:
data_final['word_tech'] = data_final['Comment'].str.contains(
    'technology|app|python|cyber|data|automation|programming|IOT|software|blockchain|server|cloud|robotics|machine learning|deep learning|hardware|artificial intelligence|neural network|algorithm|NLP|javascript|html|css|react|angular|vue|devops|git|docker|Kubernetes|AWS|Azure|Google Cloud|virtualization|containerization|hacking|encryption|firewall|cybersecurity|malware|phishing|privacy|router|5G|networking|vpn|bandwidth|big data|data science|analytics|SQL|NoSQL|data visualization|data mining|quantum computing|augmented reality|virtual reality|metaverse|cryptocurrency|startup|innovation|developer|tech stack',
    case=False, 
    na=False
).astype(int)


In [205]:
data_final['word_tech_edu'] = data_final['Comment'].str.contains(
    'technology|app|python|cyber|data|automation|programming|IOT|software|blockchain|server|cloud|robotics|machine learning|deep learning|hardware|artificial intelligence|neural network|algorithm|NLP|javascript|html|css|react|angular|vue|devops|git|docker|Kubernetes|AWS|Azure|Google Cloud|virtualization|containerization|hacking|encryption|firewall|cybersecurity|malware|phishing|privacy|router|switch|5G|networking|vpn|bandwidth|big data|data science|analytics|SQL|NoSQL|data visualization|data mining|quantum computing|augmented reality|virtual reality|metaverse|cryptocurrency|startup|innovation|developer|tech stack|edtech|e-learning|online learning|learning management system|MOOC|virtual classrooms|digital tools|gamification|interactive learning|blended learning|flipped classroom|edutainment|adaptive learning|personalized learning|digital literacy|STEM education|smart classrooms|e-books|cloud-based learning|educational apps|online assessments|collaborative learning|digital collaboration|online tutoring|distance learning|coding education|technology integration|AI tutoring systems|data-driven education|technology-enhanced learning|gamified learning|digital platforms|learning analytics|social learning platforms',
    case=False, 
    na=False
).astype(int)


In [250]:
data_final['word_workforce_dev'] = data_final['Comment'].str.contains(
    'digital|virtual|online|zoom|Microsoft Teams|COURSERA|UDEMY|LeetCode|PyCharm|HackerRank|Stack Overflow|Jupyter|TopCoder|Google Meet|Google Classroom|Moodle|Chegg|Adobe|Arduino|Raspberry pi|Magoosh|Discord|Turnitin|Grammarly|google drive|Dropbox|onedrive|blackboard|Evernote|onenote|Google Forms|bootcamp|Socrative|Google Forms|Upwork|canvas|ONLINE LEARNING|E-LEARNING|TechBloc|Upskill|Zapier|Bumble Bizz|Handshake|SAPL|SATX|Geekdom|Skillsoft|Codeup|CompTIA|Loom|CareerBuilder|Upwork|GITHUB|GITLAB|HireVue|jIRA|CONFLUENCE|Codeacademy|HackerRank|Glassdoor|AngelList|Miro|ZipRecruiter|edX|LMS|LINKEDIN|Quizlet|Kahoot|SAP|Gmail|Workday|SKILLSHARE|Notion|Pluralsight|Trello|Asana|DataCamp|Slack|AZURE|workforce development|upskilling|reskilling|skill-building platforms|online training|employee training|corporate training|job training|professional development|career advancement|digital skills|technical training|job readiness|career coaching|skill gap|vocational training|certifications|e-learning for employees|learning management system|job placement technology|workforce analytics|career development tools|AI for workforce development|soft skills training|industry-specific training|apprenticeship programs|on-the-job training|tech skills training|digital workforce|gig economy platforms',
    case=False,
    na=False
).astype(int)


In [259]:
data_final[data_final['word_workforce_dev'] == 1][data_final['comment_len'] > 30].to_csv('test2.csv')

<ipython-input-259-8cc35f01cd62>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_final[data_final['word_workforce_dev'] == 1][data_final['comment_len'] > 30].to_csv('test2.csv')


In [72]:
tech_terms = [
    'tech', 'technology', 'AI', 'artificial intelligence', 'blockchain',
    'cybersecurity', 'cloud', 'IoT', 'internet of things', 'machine learning', 
    'data', 'software', 'hardware', 'python', 'java', 'coding', 'developer', 
    'programming', 'ML', 'DL', 'deep learning', 'neural networks', 'VR', 'AR',
    'virtual reality', 'augmented reality', 'AWS', 'Microsoft', 'Apple',
    'server', 'datacenter', 'cyber', 'network', 'robotics', 'automation'
]

tech_pattern = '|'.join(tech_terms)  
data_final['tech_related'] = data_final['Comment'].str.contains(tech_pattern, case=False, na=False).astype(int)


In [108]:
data_final[data_final['web_link'] == 0][data_final['comment_len'] > 50][data_final['word_tech_edu'] == 1].to_csv('test.csv')

<ipython-input-108-8ee437258443>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_final[data_final['web_link'] == 0][data_final['comment_len'] > 50][data_final['word_tech_edu'] == 1].to_csv('test.csv')
<ipython-input-108-8ee437258443>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_final[data_final['web_link'] == 0][data_final['comment_len'] > 50][data_final['word_tech_edu'] == 1].to_csv('test.csv')


In [10]:
edu_tech_terms = [
    'e-learning', 'LMS', 'Moodle', 'Canvas', 'Coursera', 'edX', 'Khan Academy',
    'online classes', 'virtual classroom', 'digital learning', 'remote learning', 
    'educational apps', 'EdTech', 'AI in education', 'adaptive learning', 
    'gamification', 'interactive learning', 'tablets', 'iPads', 
    'Chromebooks', 'Zoom', 'Google Classroom', 'Microsoft Teams', 'STEM', 'STEAM',
    'personalized learning', 'cloud education', 'flipped classroom'
]

edu_tech_pattern = '|'.join(edu_tech_terms)
data_final['edu_tech_related'] = data_final['Comment'].str.contains(edu_tech_pattern, case=False, na=False).astype(int)

In [11]:
data_final['edu_tech_related'].value_counts()

0    1925
1      56
Name: edu_tech_related, dtype: int64

In [67]:

sa_workforce_tech_terms = [
    # General Workforce Terms
    'job training', 'upskilling', 'reskilling', 'LMS', 'online training', 
    'career development', 'certifications', 'digital skills', 'coding bootcamp',
    'cybersecurity', 'AI training', 'data analytics', 'cloud computing', 'telework',
    'remote jobs', 'virtual training', 'collaboration tools',
    
    # San Antonio-Specific Terms
    'cyber defense', 'IT jobs', 'data center', 'cloud security', 'cyber defense',
    'health IT', 'telehealth training', 'EHR systems', 'biotech', 'veteran workforce',
    'military training programs', 'DoD certifications', 'transition assistance',
    'SA Works', 'CAST Tech', 'Project QUEST', 'Workforce Solutions Alamo',
    'Port San Antonio', 'Rackspace', 'Geekdom', 'tech startups', 'incubator programs',
    
    # Broader Workforce Trends
    'machine learning', 'robotics', 'RPA', 'agile', 'DevOps', 'AI automation', 
    'STEM jobs', 'STEAM programs', 'programming languages', 'software development',
    'Microsoft Teams', 'Slack', 'Zoom', 'Google Workspace'
]

# Create a regular expression pattern from the list of terms
sa_workforce_tech_pattern = '|'.join(sa_workforce_tech_terms)  # Join terms with '|' for regex OR

# Add a new column using .str.contains() and a lambda function
data_final['workforce_tech_related'] = data_final['Comment'].str.contains(sa_workforce_tech_pattern, case=False, na=False).astype(int)

In [68]:
data_final['workforce_tech_related'].value_counts()

0    1939
1      42
Name: workforce_tech_related, dtype: int64

In [76]:
data_final[data_final['web_link'] == 0][data_final['comment_len'] > 50][data_final['tech_related'] == 1][-100:].to_csv('sample_100.csv')

<ipython-input-76-f113a2e99a63>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_final[data_final['web_link'] == 0][data_final['comment_len'] > 50][data_final['tech_related'] == 1][-100:].to_csv('sample_100.csv')
<ipython-input-76-f113a2e99a63>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_final[data_final['web_link'] == 0][data_final['comment_len'] > 50][data_final['tech_related'] == 1][-100:].to_csv('sample_100.csv')


In [12]:
data_final[data_final['web_link'] == 0][data_final['comment_len'] > 50][data_final['tech_related'] == 1][-500:-200]['workforce_tech_related']['tech_related'].value_counts()#.to_csv('sample_100_200.csv')

<ipython-input-12-89aea1ffd28a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_final[data_final['web_link'] == 0][data_final['comment_len'] > 50][data_final['tech_related'] == 1][-500:-200]['workforce_tech_related']['tech_related'].value_counts()#.to_csv('sample_100_200.csv')


KeyError: 'tech_related'